# 1. String_Distance

## Edit distance
1. Edit distance(Levenshtein distance)
2. Token edit distance
3. Jamo edit distance
4. Token - Jamo edit distance

str1, str2에 대하여 s1의 길이가 s2보다 길거나 같다고 가정함. len(s1) < len(s2)를 확인하고, s2의 길이가 더 길 경우 반대로 입력

s1에서 s2로 바뀌거나, s2에서 s1바뀌는 비용은 같음

In [1]:
_cost = 0
_cost += ('a' == 'a')
print('after cost += (a == a) : ', _cost)
_cost += ('a' != 'a')
print('after cost += (a != a) : ', _cost)
print(('a' != 'a') == 0)
print(('a' == 'a') == 1)

after cost += (a == a) :  1
after cost += (a != a) :  1
True
True


In [2]:
def levenshtein(s1, s2):
    if len(s1) < len(s2):
        return levenshtein(s2, s1)
    
    if len(s2) == 0:
        return len(s1)
    
    previous_row = range(len(s2) + 1)
    for i, c1 in enumerate(s1):
        current_row = [i + 1]
        for j, c2 in enumerate(s2):
            insertions = previous_row[j + 1] + 1
            deletions = current_row[j] + 1
            substitutions = previous_row[j] + (c1 != c2)
            current_row.append(min(insertions, deletions, substitutions))
        previous_row = current_row
    return previous_row[-1]

s1, s2에 대하여 enumerate를 통하여 iteration을 돌면서 각 unit들이 같은지 확인함. 글자 단위로 넣으면 띄어쓰기도 글자로 인식한 뒤, 각 위치의 글자들이 같은지 확인하며 Edit distance를 계산

In [3]:
test_pairs = [
    ('매직 마법 마술사', '메직 마법 마술사'),
    ('매직 마법 마술사', '메직 마법 마술사'),
    ('매직 마법 마술사', '메직 마벙 마술사'), 
    ('매직 마법 마술사', '마술사 매직 마법'),
    ('매직 마법 마술사', '마술사 매직 마버')
]

for pair in test_pairs:
    print(pair, ':', levenshtein(pair[0], pair[1]))

('매직 마법 마술사', '메직 마법 마술사') : 1
('매직 마법 마술사', '메직 마법 마술사') : 1
('매직 마법 마술사', '메직 마벙 마술사') : 2
('매직 마법 마술사', '마술사 매직 마법') : 7
('매직 마법 마술사', '마술사 매직 마버') : 7


Unit을 글자가 아니라 단어로 넣어보자

In [4]:
for pair in test_pairs:
    print(pair, ':', levenshtein(pair[0].split(), pair[1].split()))

('매직 마법 마술사', '메직 마법 마술사') : 1
('매직 마법 마술사', '메직 마법 마술사') : 1
('매직 마법 마술사', '메직 마벙 마술사') : 2
('매직 마법 마술사', '마술사 매직 마법') : 2
('매직 마법 마술사', '마술사 매직 마버') : 3


substitutions의 비용을 다르게 설정할 수 있음. c1에서 c2로 바뀌는 비용을 (c1, c2): float로 dict에 넣자

In [5]:
def levenshtein(s1, s2, cost=None):
    if len(s1) < len(s2):
        return levenshtein(s2, s1)
    
    if len(s2) == 0:
        return len(s1)
    
    # 추가된 부분
    if cost == None:
        cost = {}
        
    def get_cost(c1, c2, cost):
        return 0 if (c1 == c2) else cost.get((c1, c2), 1)
    
    previous_row = range(len(s2) + 1)
    for i, c1 in enumerate(s1):
        current_row = [i + 1]
        for j, c2 in enumerate(s2):
            insertions = previous_row[j + 1] + 1
            deletions = current_row[j] + 1
            substitutions = previous_row[j] + get_cost(c1, c2, cost)
            current_row.append(min(insertions, deletions, substitutions))
        previous_row = current_row
        
    return previous_row[-1]

print('with out cost,        서비스 --> 써비스: %.2f' % levenshtein('서비스', '써비스'))
print('with predefined cost, 서비스 --> 써비스: %.2f' % levenshtein('서비스', '써비스', {('서', '써'): 0.01}))

with out cost,        서비스 --> 써비스: 1.00
with predefined cost, 서비스 --> 써비스: 0.01


띄어쓰기가 있을 경우에는 어절 단위로, 없을 경우에는 글자 단위로 Edit distance를 계산하고, 어절 간의 거리를 임의의 함수로 정의하고 싶으면 base_distance를 argument로 받는 token_levenshtein함수를 만들면 됨

In [6]:
def token_levenshtein(s1, s2, base_distance=levenshtein, debug=False):
    t1 = set(s1.split())
    t2 = set(s2.split())
    
    cost = {(t1_i, t2_j): base_distance(t1_i, t2_j) for t1_i in t1 for t2_j in t2}
    
    if debug:
        print(cost)
        
    return levenshtein(s1.split(), s2.split(), cost)


for pair in test_pairs:
    print(pair, ':', '%d -> %d'%(levenshtein(pair[0], pair[1]), token_levenshtein(pair[0], pair[1])))

('매직 마법 마술사', '메직 마법 마술사') : 1 -> 1
('매직 마법 마술사', '메직 마법 마술사') : 1 -> 1
('매직 마법 마술사', '메직 마벙 마술사') : 2 -> 2
('매직 마법 마술사', '마술사 매직 마법') : 7 -> 2
('매직 마법 마술사', '마술사 매직 마버') : 7 -> 3


### Jamo tokenize
한글을 받아서 초/중/종성을 나눈 뒤, 한 글자를 세 글자로 이뤄진 리스트를 return하는 Jamo클래스 생성

In [7]:
class Jamo:
    kor_begin = 44032
    kor_end = 55203
    chosung_base = 588
    jungsung_base = 28
    
    chosung_list = ['ㄱ', 'ㄲ', 'ㄴ', 'ㄷ', 'ㄸ', 'ㄹ', 'ㅁ', 'ㅂ', 'ㅃ', 
            'ㅅ', 'ㅆ', 'ㅇ' , 'ㅈ', 'ㅉ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']
    
    jungsung_list = ['ㅏ', 'ㅐ', 'ㅑ', 'ㅒ', 'ㅓ', 'ㅔ', 
            'ㅕ', 'ㅖ', 'ㅗ', 'ㅘ', 'ㅙ', 'ㅚ', 
            'ㅛ', 'ㅜ', 'ㅝ', 'ㅞ', 'ㅟ', 'ㅠ', 
            'ㅡ', 'ㅢ', 'ㅣ']

    jongsung_list = [
        ' ', 'ㄱ', 'ㄲ', 'ㄳ', 'ㄴ', 'ㄵ', 'ㄶ', 'ㄷ',
            'ㄹ', 'ㄺ', 'ㄻ', 'ㄼ', 'ㄽ', 'ㄾ', 'ㄿ', 'ㅀ', 
            'ㅁ', 'ㅂ', 'ㅄ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅊ', 
            'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']

    jaum_list = ['ㄱ', 'ㄲ', 'ㄳ', 'ㄴ', 'ㄵ', 'ㄶ', 'ㄷ', 'ㄸ', 'ㄹ', 
                  'ㄺ', 'ㄻ', 'ㄼ', 'ㄽ', 'ㄾ', 'ㄿ', 'ㅀ', 'ㅁ', 'ㅂ', 
                  'ㅃ', 'ㅄ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅉ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']

    moum_list = ['ㅏ', 'ㅐ', 'ㅑ', 'ㅒ', 'ㅓ', 'ㅔ', 'ㅕ', 'ㅖ', 'ㅗ', 'ㅘ', 
                  'ㅙ', 'ㅚ', 'ㅛ', 'ㅜ', 'ㅝ', 'ㅞ', 'ㅟ', 'ㅠ', 'ㅡ', 'ㅢ', 'ㅣ']
    
    def decompose(self, c):
        i = ord(c)
        if not (self.kor_begin <= i <= self.kor_end):
            return [c]
        
        i = i - self.kor_begin
        cho = i // self.chosung_base
        jung = (i - cho * self.chosung_base) // self.jungsung_base
        jong = (i - cho * self.chosung_base - jung * self.jungsung_base)
        
        return [self.chosung_list[cho], self.jungsung_list[jung], self.jongsung_list[jong]]
    
jamo_module = Jamo()

def decompose(c):
    return jamo_module.decompose(c)

In [8]:
decompose('한')

['ㅎ', 'ㅏ', 'ㄴ']

글자의 substitution 비용을 초/중/종성을 분리한 뒤에 levenshtein거리의 1/3을 곱함으로써, 초/중/종성 차이를 모두 고려하는 levenshtein distance를 계산할 수 있음

In [9]:
def jamo_levenshtein(s1, s2):
    if len(s1) < len(s2):
        return levenshtein(s2, s1)
    
    if len(s2) == 0:
        return len(s1)
    
    def get_jamo_cost(c1, c2):
        return 0 if (c1 == c2) else levenshtein(decompose(c1), decompose(c2))/3
    
    previous_row = range(len(s2) + 1)
    for i, c1 in enumerate(s1):
        current_row = [i + 1]
        for j, c2 in enumerate(s2):
            insertions = previous_row[j + 1] + 1
            deletions = current_row[j] + 1
            substitutions = previous_row[j] + get_jamo_cost(c1, c2)
            current_row.append(min(insertions, deletions, substitutions))
        previous_row = current_row
        
    return previous_row[-1]

jamo_levenshtein('다이한민구어', '대한민국')

2.6666666666666665

In [10]:
print('%sBasic -> TOKEN -> JAMO TOKEN'%(''*35))

for pair in test_pairs:
    print(pair, ':', '%d -> %d -> %.3f'%(levenshtein(pair[0], pair[1]),
                                        token_levenshtein(pair[0], pair[1]),
                                        token_levenshtein(pair[0], pair[1], base_distance=jamo_levenshtein)))

Basic -> TOKEN -> JAMO TOKEN
('매직 마법 마술사', '메직 마법 마술사') : 1 -> 1 -> 0.333
('매직 마법 마술사', '메직 마법 마술사') : 1 -> 1 -> 0.333
('매직 마법 마술사', '메직 마벙 마술사') : 2 -> 2 -> 0.667
('매직 마법 마술사', '마술사 매직 마법') : 7 -> 2 -> 2.000
('매직 마법 마술사', '마술사 매직 마버') : 7 -> 3 -> 2.333


## Jaccard distance
자커드 거리는 unit들에 대해 set의 1 - (intersection / union) 값임
주어진 단어 s1, s2에 대하여 unit을 만드는 함수를 lambda를 통하여 정의함

    s1_set = unify(s1)
    
이 부분을 통하여 주어진 s1, s2는 unit set으로 바뀌에 되고, str을 set처리하면 글자의 set이 출력됨. set은 iterable한 대상이 입력되면 iteration을 돌면서 고유한 값을 저장함

In [11]:
def jaccard_distance(s1, s2, unitfy=lambda x:set(x), debug=False):
    if (not s1) or (not s2):
        return 1
    
    s1_set = unitfy(s1)
    s2_set = unitfy(s2)
    if debug:
        print(s1_set, s2_set)
        
    return 1 - len(s1_set.intersection(s2_set)) / len(s1_set.union(s2_set))

print(jaccard_distance('이것도요', '저것도요오', debug=True))
print(jaccard_distance('이것도요', '저것도요오', debug=True, unitfy=lambda x:set([x[i:i+2] for i in range(len(x) - 1)])))

{'도', '것', '요', '이'} {'오', '것', '요', '도', '저'}
0.5
{'것도', '도요', '이것'} {'저것', '요오', '도요', '것도'}
0.6


unify함수에 복잡한 함수를 넣을 때는 lambda와 helper함수를 함께 사용해라

char_ngram은 min_n부터 max_n길이의 character ngram을 추출하는 함수임

In [12]:
def char_ngram(x, min_n=1, max_n=3):
    ngrams = []
    max_n = min(max_n, len(x))
    
    for n in range(min_n, max_n + 1):
        for b in range(len(x) - n + 1):
            ngrams.append(x[b:b+n])
    return ngrams

print(jaccard_distance('이것도요', '저것도요오', debug=True, unitfy=lambda x:set(char_ngram(x))))

{'것도요', '이것도', '것', '도요', '이것', '요', '것도', '도', '이'} {'것도요', '요오', '오', '것도', '것', '도요', '요', '저것도', '저것', '도요오', '도', '저'}
0.6


## Cosine
Cosine distance는 Jaccard distance와 다르게, 한 unit이 몇 번 등장했는지의 횟수 정보를 이용함. 이를 위해 str s1, s2의 character들이 몇 번 등장하였는지 카운팅을 해야 함. iterable한 객체의 요소들의 카운팅은 Counter를 이용하여 쉽게 할 수 있음

In [13]:
from collections import Counter
import numpy as np

def cosine_distance(s1, s2, unitfy=lambda x: Counter(x), debug=False):
    if (not s1) or (not s2):
        return 2
    
    d1 = unitfy(s1)
    d2 = unitfy(s2)
    
    prod = 0
    for c1, f in d1.items():
        prod += (f * d2.get(c1, 0))
        
    if debug:
        print('d1 : ', d1)
        print('d2 : ', d2)
        
    return 1 - (prod / np.sqrt((sum([f**2 for f in d1.values()]) * sum([f**2 for f in d2.values()]))))


print(cosine_distance('이것도요', '저것도요오', debug=True), '\n')
print(cosine_distance('이것도요', '저것도요오', debug=True, unitfy=lambda x:Counter(char_ngram(x, min_n=1, max_n=3))))

d1 :  Counter({'이': 1, '것': 1, '도': 1, '요': 1})
d2 :  Counter({'저': 1, '것': 1, '도': 1, '요': 1, '오': 1})
0.32917960675 

d1 :  Counter({'이': 1, '것': 1, '도': 1, '요': 1, '이것': 1, '것도': 1, '도요': 1, '이것도': 1, '것도요': 1})
d2 :  Counter({'저': 1, '것': 1, '도': 1, '요': 1, '오': 1, '저것': 1, '것도': 1, '도요': 1, '요오': 1, '저것도': 1, '것도요': 1, '도요오': 1})
0.42264973081


## soynlp 사용

In [14]:
from soynlp.hangle import levenshtein, jamo_levenshtein, cosine_distance, jaccard_distance

In [15]:
print('levenshtein:\t\t\t\t%.3f'%(levenshtein('가각', '감각')))
print('jamo levenshtein:\t\t\t\t%.3f'%(jamo_levenshtein('가각', '감각')))
print('cosine w 1syllable:\t\t\t\t%.3f'%(cosine_distance('가가가각', '가가각각')))
print('cosine w 2syllable:\t\t\t\t%.3f'%(cosine_distance('가가가각', '가가각각',
        unitfy=lambda x:Counter(char_ngram(x, min_n=2, max_n=2)))))
print('jaccard w 1syllable:\t\t\t\t%.3f'%(jaccard_distance('가가각', '가가각각')))
print('jaccard w 2syllable:\t\t\t\t%.3f'%(jaccard_distance('가가각', '가가각각',
        unitfy=lambda x:set(char_ngram(x, min_n=2, max_n=2)))))

levenshtein:				1.000
jamo levenshtein:				0.333
cosine w 1syllable:				0.106
cosine w 2syllable:				0.225
jaccard w 1syllable:				0.000
jaccard w 2syllable:				0.333


# company type으로 연습

In [16]:
import sys
sys.path.append('/home/paulkim/workspace/python/Korean_NLP/WordExtraction_NounExtraction/')

from cohesion import CohesionProbability
cohesion = CohesionProbability(min_count=5)

형태는 "업태\t업종"임

In [17]:
def load_business_type_field(fname):
    with open(fname, encoding='utf-8') as f:
        twocolumns = [doc.strip().split('\t') for doc in f]
        twocolumns = [doc for doc in twocolumns if len(doc) == 2]
    business_type, business_field = zip(*twocolumns)
    return business_type, business_field

business_type, business_field = load_business_type_field('/home/paulkim/workspace/python/Korean_NLP/data/company_name/company_type_list.txt')
print(business_type[:5])
print(business_field[:5])

('도소매,서비스', '제조업,건설업', '도소매,서비스', '도소매,서비스', '서비스')
('전자상거래업, 소프트웨어개발', '컴퓨터주변기기외', '컴퓨터및주변기기', '컴퓨터및주변기기', '소프트웨어개발공급및유지보수')


normalize함수를 사용. 한글/영어/숫자/특수기호 등을 전처리하는 함수. 지워지는 글자는 빈칸으로 처리함. 두 개 이상 연속된 빈칸은 모두 지운다!

default value

    normalize(doc, english=False, number=False, punctuation=False, remove_repeat=0, remains={})
    
    > s = '이건123숫자abc영어가모두섞인 문장!! '

    > normalize(s)
    $ '이건 숫자 영어가모두섞인 문장'
    
    > normalize(s, english=True)
    $ '이건 숫자abc영어가모두섞인 문장'

In [18]:
from soynlp.hangle import normalize

business_type = [normalize(doc) for doc in business_type]
business_field = [normalize(doc) for doc in business_field]
print(business_type[:5])
print(business_field[:5])

['도소매 서비스', '제조업 건설업', '도소매 서비스', '도소매 서비스', '서비스']
['전자상거래업 소프트웨어개발', '컴퓨터주변기기외', '컴퓨터및주변기기', '컴퓨터및주변기기', '소프트웨어개발공급및유지보수']


Counter를 이용하여 빈도수가 높은 어절들을 확인해보자

서비스외, 제조외 같이 -외로 끝나는 단어들이 있음. 동시에 오탈자들도 존재함

In [19]:
from collections import Counter

business_type_tokens = Counter([token for doc in business_type for token in doc.split()])
print('num token in business type = %d'%len(business_type_tokens))
for token, freq in sorted(business_type_tokens.items(), key=lambda x:x[1], reverse=True)[:50]:
    print("%10s\t\t%d"%(token, freq))

num token in business type = 155
       서비스		7182
        제조		4298
        세관		3025
       도소매		2195
      서비스외		1155
        도매		1112
        소매		818
       부동산		815
         외		791
       제조업		767
         도		429
        통신		362
    사업서비스외		352
       제조외		309
        건설		297
      운수관련		249
        운보		223
      서비스업		208
      도소매외		196
       건설업		156
       통신업		144
        운수		119
       도매업		116
       써비스		113
       소매외		101
       운수업		100
      부동산업		83
      통신업외		83
       소매업		61
     전기통신업		61
        보관		55
      전기가스		51
   운수관련서비스		45
       운보외		44
         및		42
       음숙외		40
     다단계판매		39
       도매외		38
   부동산업부동산		37
        음숙		36
      기타관련		35
     상품중개업		34
       건설외		33
      도소매업		30
      부동산외		27
      사업관련		26
         업		26
        농업		26
      제조업외		25
       비영리		24


CohesionProbability를 학습

In [20]:
cohesion.train(business_type)

inserting subwords into L: done
num subword = 356
num subword = 229 (after pruning with min count 5)


In [21]:
print("%12s\t\t(freq, cohesion_l, cohesion_r)\n%s"%('단어', '-'*50))
for token, freq in sorted(business_type_tokens.items(), key=lambda x:x[1], reverse=True):
    cohesion_l = cohesion.get_cohesion(token)
    freq = cohesion.L.get(token, 0)
    
    if cohesion_l < 0.1 or freq < 10 or len(token) == 1:
        continue
    print('%12s\t\t(%4d, %.3f)' % (token, freq, cohesion_l))

          단어		(freq, cohesion_l, cohesion_r)
--------------------------------------------------
         서비스		(8595, 0.998)
          제조		(5436, 0.998)
          세관		(3025, 1.000)
         도소매		(2439, 0.766)
        서비스외		(1155, 0.512)
          도매		(1277, 0.307)
          소매		( 980, 0.998)
         부동산		( 971, 1.000)
         제조업		( 808, 0.385)
          통신		( 603, 1.000)
      사업서비스외		( 352, 0.955)
         제조외		( 309, 0.238)
          건설		( 493, 0.998)
        운수관련		( 294, 0.710)
          운보		( 274, 0.334)
        서비스업		( 225, 0.297)
        도소매외		( 196, 0.361)
         건설업		( 160, 0.569)
         통신업		( 227, 0.614)
          운수		( 527, 0.643)
         도매업		( 117, 0.168)
         써비스		( 113, 1.000)
         소매외		( 101, 0.321)
         운수업		( 101, 0.351)
        부동산업		( 120, 0.498)
        통신업외		(  83, 0.516)
         소매업		(  61, 0.249)
       전기통신업		(  61, 0.859)
          보관		(  56, 0.903)
        전기가스		(  51, 0.769)
     운수관련서비스		(  45, 0.616)
         운보외		(  44, 0.232)
        

초성/중성/종성을 분해하는 jamo_levenshtein을 이용하기 위해 string_distance에서 불러오자

In [22]:
from soynlp.hangle import jamo_levenshtein

jamo_levenshtein('아니야', '아니얌')

0.3333333333333333

match함수는 주어진 token에 대해 현재까지 단어로 알려진 dict를 이용해, 길이가 2보다 긴 단어들의 조합으로 이뤄진 token을 분해함

    {
        '도소매' : 100,
        '서비스' : 50,
        '소매' : 30
    }
    
이 알려진 dictionary일 경우, '도소매서비스판매'에 대해, 빈도수가 가장 높은 단어 '도소매'가 들어있기 때문에 token을 아래와 같이 변형함

    '도소매서버스판매' --> '[도소매]서비스판매'
    
[, ]로 경계를 설정한 이유는 이후 '소매'라는 단어가

    '[도소매]서비스판매' --> '[도[소매]]서비스판매'
    
처럼 변경될 것을 막기 위함임

사전에 있는 모든 단어에 대해 확인이 끝나면, [, ]부분에 split()을 하여 token을 단어로 나눔. 이를 통해 여러 업태의 합성어로 이뤄진 업태를 토크나이징함

In [23]:
def match(token, dictionary):
    for word, _ in sorted(dictionary.items(), key=lambda x:x[1], reverse=True):
        if (word in token) and ( ('[%s'%word in token) == False ) and ( ('%s]'%word in token) == False):
            token = token.replace(word, '[%s]' % word)
            
    # 제조 서비스 업 -> 제조 서비스
    token = token.replace('[', ']').replace(']]',']')
    token = ' '.join([t for t in token.split(']') if len(t) >= 2])
    return token.strip()

for token in ['도소매서비스판매', '금융서비스']:
    print('%s --> %s' % (token, match(token, {'도소매':100, '서비스':50, '소매':30})))

도소매서비스판매 --> 도소매 서비스 판매
금융서비스 --> 금융 서비스


find_similars함수는 dictionary에 들어있는 단어들 중에서 token과 jamo_leveshtein거리가 <= 0.5 이하인 단어를 찾는 함수임. 거리값은 1/3단위로 변하기 때문에 거리 <=0.5는 초/중/종성 중 하나 이상 다르지 않다는 것!!!을 기억해야 함

dictionary 안에 두 개 이상으로 유사한 단어가 있다면 빈도수가 가장 높은 단어를 선택함. 만약 비슷한 단어가 없으면 None을 리턴함

In [24]:
def find_similars(token, dictionary):
    if not dictionary:
        return None
    
    similars = {word:jamo_levenshtein(token, word) for word in dictionary.keys()}
    min_distance = min(similars.values())
    if min_distance > 0.5:
        return None
    
    similars = {word:dist for word, dist in similars.items() if dist == min_distance}
    return sorted(similars.keys(), key=lambda x:dictionary.get(x, 0), reverse=True)[0]

print(find_similars('써비스', {'서비스':100}))
print(find_similars('자비스', {'서비스':100}))

서비스
None


업태의 표현들을 제대로 된 단어들과 노이즈로 나누는 작업을 수행함

business_type_dictionary = {}는 빈 dict로 시작하여, 제대로 된 단어들을 체울 것임.
business_type_noisy = {}는 빈 dict로 시작하여, 노이즈들을 추가할 것임

business_type_dictionary는 {단어:빈도수} 형태로 입력할 것임. business_type_noisy은 {오탈자:정자} 형태로 입력할 것임.  {'써비스', : '서비스'} 임

passwords는 단어로 알려진 업태임. passwords = ['작업', '도매', '도소매', '금융']로 네 개의 단어만 알고 있다고 가정함

    for password in passwords:
        business_type_dictionary[password] = business_type_tokens[password]
        
correct_noise 함수는 tokens, dictionary, noisy, 세 가지 dict를 받아서, dictionary, noisy를 return해야 함

correct_noise 함수는 token을 여섯가지 경우로 나눠서 생각함
1. 길이가 1보다 작은 단어들로 제거
2. "서비스외 --> 서비스"로 고치는 경우로, 뒤의 한 글자를 제외하면 이미 dictionary에 등록되어 있는 단어들임
3. "써비스업외 --> 서비스 + 업외"처럼 나눠지는 경우로, 뒤의 두글자를 제외하면 단어가 dictionary에 등록되어 있지만, 뒤의 두글자는 등록이 되어있지 않는 경우
4. "도매서비스 --> 도매 + 서비스"처럼 두 개 이상의 업태 단어가 합성된 경우. 앞의 match함수를 이용하여 띄어쓰기가 추가가 되면 이는 합성어로 판단하고 수정
5. "써비스 -> 서비스"처럼 dictionary에 jamo_levenshtein 거리가 0.5 이하인 단어가 존재하는 경우. 이 때에는 dictionary에 존재하는 단어로 치환
6. 그 외의 단어는 dictionary에 등록하여 다음 단어로 넘어감

correct_noise함수는 주어진 tokens에 대하여 빈도수 기준으로 내림정렬을 하고, 빈도수가 높은 단어부터 위 경우 중 어떠한 경우에 해당하는지 살펴봄

빈도수가 높은 단어부터 유사하거나 합성어가 아닌 경우, 올바른 단어로 추가하는 것임

In [25]:
import sys

business_type_dictionary = {} # word: freq
business_type_noisy = {} # noisy word to corrected word

# exception
passwords = ['소매', '도매', '도소매', '금융']
for password in passwords:
    business_type_dictionary[password] = business_type_tokens[password]

def correct_noise(tokens, dictionary, noisy):
    for num, (token, freq) in enumerate(sorted(tokens.items(), key=lambda x:x[1], reverse=True)):
        if (num + 1) % 100 == 0:
            sys.stdout.write('\r%d in %d' % (num + 1, len(tokens)))
        
        if len(token) <= 1:
            business_type_noisy[token] = ''
            continue

        # check 서비스외 = 서비스 + 외
        if (token[:-1] in dictionary):
            noisy[token] = token[:-1]
            continue

        # check 서비스업외 = 서비스 + 업외
        if (len(token) > 2) and (token[:-2] in dictionary) and ((token[-2:] in dictionary) == False):
            noisy[token] = token[:-2]
            continue

        # check 도매서비스 = 도매 + 서비스
        matched_token = match(token, dictionary)
        if ' ' in matched_token:
            # check 운보써비스 -> [운보]써비스 -> [운보][서비스]
            spells = [find_similars(token, business_type_dictionary) for token in matched_token.split()]
            matched_token = ' '.join([s if s else m for m, s in zip(matched_token.split(), spells)])
            noisy[token] = matched_token
            continue

        # check 써비스 -> 서비스
        most_similar = find_similars(token, dictionary)
        if (most_similar != None) and (most_similar != token):
            noisy[token] = most_similar
            continue

        # Add token into dictionary
        dictionary[token] = freq
    
    print('\rdone')        
    return dictionary, noisy


business_type_dictionary, business_type_noisy = correct_noise(business_type_tokens, business_type_dictionary, business_type_noisy)
print('all tokens are parsed?', len(business_type_dictionary) + len(business_type_noisy) == len(business_type_tokens))

donein 155
all tokens are parsed? True


business_type_dictionary에 등록된 단어임. '음숙외'와 같은 단어가 사전에 들어있음. 이는 '음숙'이 36번으로 '음숙외'보다 덜 등장했기 때문임

In [26]:
result = (sorted(business_type_dictionary.items(), key=lambda x:x[1], reverse=True))
for i, business_type in enumerate(result):
    print("#%2d:  "% i, business_type)

# 0:   ('서비스', 7182)
# 1:   ('제조', 4298)
# 2:   ('세관', 3025)
# 3:   ('도소매', 2195)
# 4:   ('도매', 1112)
# 5:   ('소매', 818)
# 6:   ('부동산', 815)
# 7:   ('통신', 362)
# 8:   ('건설', 297)
# 9:   ('운수관련', 249)
#10:   ('운보', 223)
#11:   ('운수', 119)
#12:   ('보관', 55)
#13:   ('전기가스', 51)
#14:   ('음숙외', 40)
#15:   ('다단계판매', 39)
#16:   ('음숙', 36)
#17:   ('기타관련', 35)
#18:   ('상품중개업', 34)
#19:   ('사업관련', 26)
#20:   ('농업', 26)
#21:   ('비영리', 24)
#22:   ('서비', 21)
#23:   ('자유직업', 21)
#24:   ('사회', 20)
#25:   ('해운외', 19)
#26:   ('운송', 19)
#27:   ('도난방지기및감시장치', 17)
#28:   ('연구개발컨설팅', 16)
#29:   ('유리및', 14)
#30:   ('유리제품', 14)
#31:   ('정보동신', 13)
#32:   ('교육', 13)
#33:   ('작물재배', 9)
#34:   ('의료학회', 8)
#35:   ('사단법인', 8)
#36:   ('출판', 8)
#37:   ('숙박업', 7)
#38:   ('음식업', 7)
#39:   ('보건업', 5)
#40:   ('학교', 5)
#41:   ('자동자제외', 4)
#42:   ('의료업', 2)
#43:   ('데이터베이스', 2)
#44:   ('판매', 2)
#45:   ('정화조청소', 2)
#46:   ('브랜드개발', 2)
#47:   ('광고기획', 2)
#48:   ('금융', 1)
#49:   ('화물취급업', 1)
#50:   ('소도매', 1)
#51:   ('무선전화업

business_type_noisy에 들어있는 단어들이 어떻게 오탈자가 수정되었는 확인

    건설제조업건설서비스(1) -> 건설 제조 건설 서비스
    
처럼 여러 개의 합성어로 이뤄진 업태도 분해가 가능함

In [27]:
for noisy_token, corrected in sorted(business_type_noisy.items(), key=lambda x:business_type_tokens.get(x[0], 0), reverse=True):
    print('%10s (%d)\t-->\t%s' % (noisy_token, business_type_tokens[noisy_token], corrected))

      서비스외 (1155)	-->	서비스
         외 (791)	-->	
       제조업 (767)	-->	제조
         도 (429)	-->	
    사업서비스외 (352)	-->	사업 서비스
       제조외 (309)	-->	제조
      서비스업 (208)	-->	서비스
      도소매외 (196)	-->	도소매
       건설업 (156)	-->	건설
       통신업 (144)	-->	통신
       도매업 (116)	-->	도매
       써비스 (113)	-->	서비스
       소매외 (101)	-->	소매
       운수업 (100)	-->	운수
      부동산업 (83)	-->	부동산
      통신업외 (83)	-->	통신
       소매업 (61)	-->	소매
     전기통신업 (61)	-->	전기 통신
   운수관련서비스 (45)	-->	운수관련 서비스
       운보외 (44)	-->	운보
         및 (42)	-->	
       도매외 (38)	-->	도매
   부동산업부동산 (37)	-->	부동산 부동산
       건설외 (33)	-->	건설
      도소매업 (30)	-->	도소매
      부동산외 (27)	-->	부동산
         업 (26)	-->	
      제조업외 (25)	-->	제조
     법무서비스 (24)	-->	법무 서비스
    서비스도소매 (23)	-->	서비스 도소매
   자동차제조업등 (20)	-->	자동차 제조 업등
   식품제조가공업 (19)	-->	식품 제조 가공업
     서비스업외 (17)	-->	서비스
    도소매서비스 (17)	-->	도소매 서비스
       금융업 (16)	-->	금융
    사업서비스업 (16)	-->	사업 서비스
        음식 (15)	-->	음숙
       통신외 (14)	-->	통신
    금융및보험업 (14)	-->	금융 및보험업
     사업서비스 (13)	-->	사업 서비스
      

동일한 작업을 업종에 대해서도 수행

In [28]:
business_field_tokens = Counter([token for doc in business_field for token in doc.split()])

business_field_dictionary = {}
business_field_noisy = {}

passwords = ['소매', '도매','도소매', '금융']
for password in passwords:
    business_field_dictionary[password] = business_field_tokens[password]
    
business_field_dictionary, business_field_noisy = correct_noise(business_field_tokens, business_field_dictionary, business_field_noisy)

done in 1237


In [29]:
sorted(business_type_dictionary.items(), key=lambda x:x[1], reverse=True)

[('서비스', 7182),
 ('제조', 4298),
 ('세관', 3025),
 ('도소매', 2195),
 ('도매', 1112),
 ('소매', 818),
 ('부동산', 815),
 ('통신', 362),
 ('건설', 297),
 ('운수관련', 249),
 ('운보', 223),
 ('운수', 119),
 ('보관', 55),
 ('전기가스', 51),
 ('음숙외', 40),
 ('다단계판매', 39),
 ('음숙', 36),
 ('기타관련', 35),
 ('상품중개업', 34),
 ('사업관련', 26),
 ('농업', 26),
 ('비영리', 24),
 ('서비', 21),
 ('자유직업', 21),
 ('사회', 20),
 ('해운외', 19),
 ('운송', 19),
 ('도난방지기및감시장치', 17),
 ('연구개발컨설팅', 16),
 ('유리및', 14),
 ('유리제품', 14),
 ('정보동신', 13),
 ('교육', 13),
 ('작물재배', 9),
 ('의료학회', 8),
 ('사단법인', 8),
 ('출판', 8),
 ('숙박업', 7),
 ('음식업', 7),
 ('보건업', 5),
 ('학교', 5),
 ('자동자제외', 4),
 ('의료업', 2),
 ('데이터베이스', 2),
 ('판매', 2),
 ('정화조청소', 2),
 ('브랜드개발', 2),
 ('광고기획', 2),
 ('금융', 1),
 ('화물취급업', 1),
 ('소도매', 1),
 ('무선전화업', 1),
 ('제작', 1),
 ('해운업', 1),
 ('숙박', 1),
 ('의료', 1),
 ('건서업', 1)]

In [30]:
for noisy_token, corrected in sorted(business_field_noisy.items(), key=lambda x:business_field_tokens.get(x[0], 0), reverse=True):
    print('%16s (%d)\t -> \t%s'%(noisy_token, business_field_tokens[noisy_token], corrected))

          다단계판매원 (188)	 -> 	다단계판매
            화분임대 (184)	 -> 	화분 임대
            광고선전 (114)	 -> 	광고
      인쇄출판디자인기획외 (91)	 -> 	인쇄 출판 디자인 기획외
  산업단지의개발조성및분양임대 (80)	 -> 	산업단지의개발조성및분양 임대
           광고대행외 (75)	 -> 	광고대행
       소프트웨어개발공급 (69)	 -> 	소프트웨어개발
          개발및판매외 (67)	 -> 	개발 및판매외
          다단계후원수 (65)	 -> 	다단계 후원수
      소프트웨어개발및공급 (64)	 -> 	소프트웨어개발 및공급
            컨설팅외 (62)	 -> 	컨설팅
            수세미외 (60)	 -> 	수세미
           의약품제조 (55)	 -> 	의약품
    소프트웨어프로그램개발외 (54)	 -> 	소프트웨어 프로그램 개발
           디지털인쇄 (53)	 -> 	디지털 인쇄
        소프트웨어개발외 (52)	 -> 	소프트웨어개발
          종합유선방송 (48)	 -> 	종합 유선 방송
           옵셋인쇄외 (46)	 -> 	옵셋 인쇄
         소프트웨어자문 (43)	 -> 	소프트웨어
         다단계후원수당 (40)	 -> 	다단계 후원수당
              전기 (37)	 -> 	전시
          유선통신장비 (37)	 -> 	유선 통신장비
          광고물제작외 (33)	 -> 	광고 제작
          위탁대리판매 (33)	 -> 	위탁대리 판매
            유선방송 (32)	 -> 	유선
             인쇄외 (31)	 -> 	인쇄
         기술검사서비스 (29)	 -> 	기술검사 서비스
           주차장운영 (28)	 -> 	주차장
            개발용역 (28)	 -> 	개발
          

         국제회의기획외 (3)	 -> 	국제회의 기획
          판촉물잡자재 (3)	 -> 	판촉물 잡자재
           유선방송업 (3)	 -> 	유선 방송업
           기업컨설팅 (3)	 -> 	기업 컨설팅
            어학서적 (3)	 -> 	어학 서적
         산업용인쇄기기 (3)	 -> 	산업 인쇄 기기
             인테넷 (3)	 -> 	인터넷
         가전생활용품외 (3)	 -> 	가전생활용품
       방송채널사용사업외 (3)	 -> 	방송채널사용사업
            조명기구 (3)	 -> 	조명기기
           출판물판매 (3)	 -> 	출판 판매
           자문및개발 (3)	 -> 	자문 개발
           금융전산망 (3)	 -> 	금융 전산망
          경영컨설팅외 (3)	 -> 	경영 컨설팅
          홈페이지제작 (3)	 -> 	홈페이지 제작
            행사기획 (3)	 -> 	행사 기획
            교육사업 (3)	 -> 	교육
         전기통신공사외 (3)	 -> 	전기통신공사
            시계제작 (3)	 -> 	시계 제작
            컴퓨터및 (3)	 -> 	컴퓨터
           가스수리업 (3)	 -> 	가스 수리
          냉난방기기외 (3)	 -> 	냉난방기
          자전거부품외 (3)	 -> 	자전거 부품
            병원홍보 (3)	 -> 	병원 홍보
           마케팅대행 (3)	 -> 	마케팅
       가정용플라스틱제품 (3)	 -> 	가정용 플라스틱 제품
유선통신장치무선통신방송및응용장치 (3)	 -> 	유선 통신 장치무선 통신 방송및응용장치
             통신판 (3)	 -> 	통신
             은행업 (3)	 -> 	은행
       자전거및자전거부품 (3)	 -> 	자전거 자전거 부품
          